In [ ]:
import subprocess
import json

#Configure Clingo parameters

clingo_path = 'C:\\Users\\Alex\\Documents\\MSc Computer Science\\ASP\\clingo.exe'
clingo_options = ['--outf=2','-n 0']
clingo_command = [clingo_path] + clingo_options

#Solve the ruleset.
def solve(program):
    input = program.encode()
    process = subprocess.Popen(clingo_command, stdin=subprocess.PIPE, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    output, error = process.communicate(input)
    result = json.loads(output.decode())
    if result['Result'] == 'SATISFIABLE':
        return [value['Value'] for value in result['Call'][0]['Witnesses']]
    else:
        return None

In [ ]:
def write_problem(constraints):
    all_rules = [['rules\\kick_placement.lp', 'rules\\kick_con_1.lp', 'rules\\kick_con_2.lp'], \
                 ['rules\\snare_placement_exp.lp', 'rules\\snare_placement_conv.lp'], \
                 ['rules\\kick_snare_con_1.lp'], \
                 ['rules\\hat_placement_exp.lp', 'rules\\hat_placement_conv.lp' , 'rules\\hat_con_1.lp', 'rules\\hat_con_2.lp'], \
                ['rules\\perc_placement.lp', 'rules\\perc_con_1.lp', 'rules\\perc_con_2.lp'], \
                ['rules\\gsnare_placement.lp', 'rules\\gsnare_con_1.lp', 'rules\\gsnare_con_2.lp']]
    rules = ['rules\\time.lp']
    i = 0
    for row in all_rules:
        for j in range(0, len(row)):
            if constraints[i][j]:
                rules.append(all_rules[i][j])
        i += 1
    with open('rules\\problem.lp', 'w') as outfile:
        for fname in rules:
            with open(fname) as infile:
                outfile.write(infile.read())    

In [ ]:
#Solve the problem in Clingo, specifying what kind of pattern will be generated.
#Print the number of solutions.
def generate_solutions(constraints, user_input):
    write_problem(constraints)
    if user_input:     
        with open('rules\\problem.lp', 'a') as outfile:
            outfile.write('\n'.join(user_input) + '\n')
    problem = open('rules\\problem.lp', 'r').read()
    solutions = solve(problem)
    #Print the number of patterns found.
    print(str(len(solutions)) + " patterns have been found.\n")

    return solutions

In [ ]:
#Represent hit_list as a table.

import numpy as np
import matplotlib.pyplot as plt
#import random
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import FixedLocator

def print_hits(pattern_index, hit_list):

    X = np.ones((5, 16))

    for i in hit_list:
        if i[0] == 'k':
            X[4][int(i[1])-1] = 0
        if i[0] == 's':
            X[3][int(i[1])-1] = 0.2
        if i[0] == 'g':
            X[2][int(i[1])-1] = 0.4
        if i[0] == 'h':
            X[1][int(i[1])-1] = 0.6
        if i[0] == 'p':
            X[0][int(i[1])-1] = 0.8

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(X, cmap=plt.get_cmap('gray'))

    y_labels = ['Percussion', 'Hat', 'Ghost Snare', 'Snare', 'Kick']
    ax.set_xticks(list(range(0, 16)))
    ax.set_yticks(list(range(0, 5)))
    ax.set_yticklabels(y_labels)
    ax.set_xticklabels(list(range(1, 17)))
    
    minor_xlocator = AutoMinorLocator(2)
    minor_ylocator = FixedLocator([0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5, 13.5, 14.5, 15.5, 16.5])
    ax.xaxis.set_minor_locator(minor_xlocator)
    ax.yaxis.set_minor_locator(minor_ylocator)
    ax.grid(which='minor', color='0.4', linestyle='dashed')
    ax.set_title("Pattern " + str(pattern_index), fontsize=18)

    plt.show()

In [ ]:
#Create MIDI representation of the drum pattern.

from midiutil import MIDIFile

#offset needed to convert to MIDI time.
offset = 0.25
track    = 0
channel  = 0
duration = 0.25 # In beats
tempo    = 174  # In BPM
volume   = 100  # 0-127, as per the MIDI standard

#Write a MIDI file for a set of hits with a given file name.
def write_midi(hit_list, file_name):
    MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
                          # automatically)
    MyMIDI.addTempo(track, 0, tempo)

    #Add kicks to middle C, snares to C#3 and hats to D3.
    for i in hit_list:
        if i[0] == 'k':
            MyMIDI.addNote(track, channel, 60, float(i[1])/4-offset, duration, volume)
        if i[0] == 's':
            MyMIDI.addNote(track, channel, 61, float(i[1])/4-offset, duration, volume)
        if i[0] == 'g':
            MyMIDI.addNote(track, channel, 61, float(i[1])/4-offset, duration, 40)
        if i[0] == 'h':
            MyMIDI.addNote(track, channel, 62, float(i[1])/4-offset, duration, volume)
        if i[0] == 'p':
            MyMIDI.addNote(track, channel, 63, float(i[1])/4-offset, duration, volume)
            
    with open(file_name, "wb") as output_file:
        MyMIDI.writeFile(output_file)

In [ ]:
#Generate n random patterns from the answer set, depending on the rules chosen and any user input. 
#Print and store solutions as MIDI files. Defaults to 1 pattern with no user input.
from random import randint

def generate_patterns(constraints, n = 1, user_input = None):
    solutions = generate_solutions(constraints, user_input)
    for i in range(1, n+1):
        rand_index = randint(0, len(solutions)-1)
        rand_solution = solutions[rand_index][:]
        #Converting the random solution into a (hit, quarter-beat) list of type (char, char).
        hit_list = []
        for hit in rand_solution:
            hit_list.append(hit[hit.find("(")+1:hit.find(")")].split(","))
        
        print_hits(rand_index, hit_list)
        print("\n")
        write_midi(hit_list, "test_pattern_" + str(i) +".mid")

In [ ]:
import timeit

start = timeit.default_timer()

#constraints is a boolean list indicating which rules to include in the problem.
#It is in the following format:
# constraints = [[kick placement, kick mild constraints, kick moderate constraints], \
#               [snare placement with mild constraints, snare placement with moderate constraints],
#               [kick snare additional constraints],
#               [hat placement w/ mild constraints, hat placement w/ moderate contraints, hat additional mild constraints, hat additional moderate constraints],
#               [percussion placement, percussion mild constraints, percussion moderate constraints]
#               [ghost snare placement, ghost snare mild constraints, ghost snare moderate constraints]]

constraints = [ [True, True, True], \
                [False, True], \
                [True], \
                [False, True, True, True], \
                [True, True, True], \
                [True, True, True] ]

#Patterns will be filled out around any user input given to the program.
user_input = ["chooseHit(k, 3).", "chooseHit(s, 5).", "chooseHit(s, 13)."]

generate_patterns(constraints, 1, user_input)

stop = timeit.default_timer()

runtime = stop - start

print("runtime: " + str(round(runtime, 3)) + " seconds") 